Trying to vectorize some of Ivan's codes.

# Performance
need to figure out relative performance of some of numpy's routines for matrix transformations

In [1]:
import numpy as np

In [7]:
A = np.random.rand(50,30)
B = np.random.rand(30)

## Dot product

In [8]:
%timeit np.dot(A,B)

2.12 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
%timeit np.matmul(A,B.reshape(30,1))

2.96 µs ± 347 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
C=B.reshape(30,1)
%timeit np.matmul(A,C)


2.29 µs ± 116 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [16]:
%timeit np.einsum('ij,j',A,B)

6.23 µs ± 340 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Product

In [18]:
%timeit (A*B).shape

5.31 µs ± 1 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [22]:
%timeit np.multiply(A,B)

4.36 µs ± 198 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [21]:
%timeit np.einsum('ij,j->i',A,B)

5.38 µs ± 150 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [29]:
def for_prod(A,B):
    
    l1, l2 =A.shape
    C=np.ones(A.shape)
    for i in range(l1):
        for k in range(l2):
            C[i,k] = A[i,k]*B[k]

%timeit for_prod(A,B)

1.08 ms ± 14.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Using out
Not a good idea

In [30]:
%timeit np.multiply(A,B)

5.34 µs ± 976 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [36]:
%timeit np.multiply(A,B, out = C)

33.1 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [39]:
C= np.einsum('ij,j',A,B)
%timeit np.einsum('ij,j',A,B, out=C)

35.6 µs ± 1.25 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [40]:
C= np.einsum('ij,j->j',A,B)
%timeit np.einsum('ij,j->j',A,B, out=C)

38.6 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Vectorize Ivan's code
## Original for loops


In [45]:
def for_loops_1(nr, nz, N1, N2):
    #for test only
    drv = dz = 2
    R1 =15
    R2 =20
    LAMBDA = 5
    
    #code of interest:
    
    J=np.zeros((nr, nz))
    for ir in range(nr):
        r = (0.5+ir)*drv
        for iz in range(nz):
            z = (iz+0.5) *dz
            s=0
            for k1 in range(N1):
                rc = R1 + (k1+0.5) *(R2-R1)/N1
                for k2 in range(N2):
                    fi = (k2 +0.5) * np.pi/N2
                    r0 = np.sqrt(
                            z*z + r*r + rc*rc  - 2*r*rc*np.cos(fi))
                    s += rc* z * np.exp(-r0/LAMBDA) /(
                            LAMBDA * np.power(r0, 3))
            J[ir, iz] = ((R2-R1)/N1) * (np.pi/N2) * s * 2 /np.pi
    return(J)


## 1. Make loops onto matrices, but still loop over elements

In [50]:
def for_loops_1b(nr, nz, N1, N2):
    #for test only
    drv = dz = 2
    R1 =15
    R2 =20
    LAMBDA = 5
    
    #code of interest:
    
    Mr = np.zeros(nr)
    Mz = np.zeros(nz)
    Mrc = np.zeros(N1)
    Mfi = np.zeros(N2)
    
    J=np.zeros((nr, nz))
    for ir in range(nr):
        r = (0.5+ir)*drv
        Mr[ir] = r
        for iz in range(nz):
            z = (iz+0.5) *dz
            Mz[iz]=z
            s=0
            for k1 in range(N1):
                rc = R1 + (k1+0.5) *(R2-R1)/N1
                Mrc[k1] = rc
                for k2 in range(N2):
                    fi = (k2 +0.5) * np.pi/N2
                    Mfi[k2] = fi
                    r0 = np.sqrt(
                            z*z + r*r + rc*rc  - 2*r*rc*np.cos(fi))
                    s += rc* z * np.exp(-r0/LAMBDA) /(
                            LAMBDA * np.power(r0, 3))
            J[ir, iz] = ((R2-R1)/N1) * (np.pi/N2) * s * 2 /np.pi
    return(J, Mr, Mz, Mrc, Mfi)



In [53]:
def for_loops_2(nr, nz, N1, N2):
    #for test only
    drv = dz = 2
    R1 =15
    R2 =20
    LAMBDA = 5
    
    #code of interest:
    Mr =  drv * (np.arange(nr)+0.5)
    Mz = dz * (np.arange(nz)+0.5)
    Mrc =  R1 + ((R2-R1)/N1) * (np.arange(N1)+0.5) 
    Mfi = (np.pi/N2) * (np.arange(N2)+0.5)
    
    J=np.zeros((nr, nz))
    for ir in range(nr):
        r = Mr[ir]
        for iz in range(nz):
            z = Mz[iz]
            s=0
            for k1 in range(N1):
                rc = Mrc[k1]
                for k2 in range(N2):
                    fi = Mfi[k2]
                    
                    r0 = np.sqrt(
                            z*z + r*r + rc*rc  - 2*r*rc*np.cos(fi))
                    s += rc* z * np.exp(-r0/LAMBDA) /(
                            LAMBDA * np.power(r0, 3))
            J[ir, iz] = ((R2-R1)/N1) * (np.pi/N2) * s * 2 /np.pi
    return(J, Mr, Mz, Mrc, Mfi)

In [54]:
J1 = for_loops_1b(2,3,4,5)
J2 = for_loops_2(2,3,4,5)

print(J1[1])
print()
print(J2[1])

[1. 3.]

[1. 3.]


In [56]:
print(J1[0]-J2[0])

[[0. 0. 0.]
 [0. 0. 0.]]


## 2. Build r0 as matrix Mr0

In [89]:
def for_loops_3a(nr, nz, N1, N2):
    #for test only
    drv = dz = 2
    R1 =15
    R2 =20
    LAMBDA = 5
    
    #code of interest:
    Mr =  drv * (np.arange(nr)+0.5)
    Mz = dz * (np.arange(nz)+0.5)
    Mrc =  R1 + ((R2-R1)/N1) * (np.arange(N1)+0.5) 
    Mfi = (np.pi/N2) * (np.arange(N2)+0.5)
    
    Mr0 = np.zeros((nr, nz, N1, N2))
    Ms = np.zeros((nr, nz))
    
    J=np.zeros((nr, nz))
    for ir in range(nr):
        r = Mr[ir]
        for iz in range(nz):
            z = Mz[iz]
            s=0
            for k1 in range(N1):
                rc = Mrc[k1]
                for k2 in range(N2):
                    fi = Mfi[k2]
                    
                    r0 = np.sqrt(
                            z*z + r*r + rc*rc - 2*r*rc*np.cos(fi))
                    #r0 = 2*r*rc*np.cos(fi)
                    Mr0[ir,iz,k1, k2] = r0
                    #s += rc* z * np.exp(-r0/LAMBDA) /(
                     #       LAMBDA * np.power(r0, 3))
           # Ms[ir,iz] = s
           # J[ir, iz] = ((R2-R1)/N1) * (np.pi/N2) * s * 2 /np.pi
    return(Mr0)

In [90]:
def vec_3a(nr, nz, N1, N2):
    #for test only
    drv = dz = 2
    R1 =15
    R2 =20
    LAMBDA = 5
    
    #code of interest:
    Mr =  drv * (np.arange(nr)+0.5)
    Mz = dz * (np.arange(nz)+0.5)
    Mrc =  R1 + ((R2-R1)/N1) * (np.arange(N1)+0.5) 
    Mfi = (np.pi/N2) * (np.arange(N2)+0.5)
    
    Ms = np.zeros((nr, nz))
    
    Mr00 = Mr.reshape((nr,1,1,1)) * Mrc.reshape((1,1,N1,1))
    Mr01 = Mr00 * np.cos(Mfi).reshape((1,1,1,N2))
    Mr00 = (Mz**2).reshape(1,nz,1,1) - 2 * Mr01
    Mr01 = Mr00 + (Mr**2).reshape((nr,1,1,1)) 
    Mr00 = Mr01 + (Mrc**2).reshape((1,1,N1,1))
    Mr0 = np.sqrt(Mr00)
                    
    return(Mr0)

In [91]:
Mr00 = for_loops_3a(2,3,4,5)
Mr01 = vec_3a(2,3,4,5)

In [92]:
Mr01.shape

(2, 3, 4, 5)

In [93]:
Mr00.shape

(2, 3, 4, 5)

In [94]:
print(np.max(Mr00-Mr01))
print(np.min(Mr00-Mr01))

3.552713678800501e-15
0.0


## 3. Build s as matrix MS

In [95]:
def for_loops_4(nr, nz, N1, N2):
    #for test only
    drv = dz = 2
    R1 =15
    R2 =20
    LAMBDA = 5
    
    #code of interest:
    Mr =  drv * (np.arange(nr)+0.5)
    Mz = dz * (np.arange(nz)+0.5)
    Mrc =  R1 + ((R2-R1)/N1) * (np.arange(N1)+0.5) 
    Mfi = (np.pi/N2) * (np.arange(N2)+0.5)
    
    Mr0 = np.zeros((nr, nz, N1, N2))
    Ms = np.zeros((nr, nz))
    
    J=np.zeros((nr, nz))
    for ir in range(nr):
        r = Mr[ir]
        for iz in range(nz):
            z = Mz[iz]
            s=0
            for k1 in range(N1):
                rc = Mrc[k1]
                for k2 in range(N2):
                    fi = Mfi[k2]
                    
                    r0 = np.sqrt(
                            z*z + r*r + rc*rc - 2*r*rc*np.cos(fi))
                    #r0 = 2*r*rc*np.cos(fi)
                    Mr0[ir,iz,k1, k2] = r0
                    s += rc* z * np.exp(-r0/LAMBDA) /(
                            LAMBDA * np.power(r0, 3))
            Ms[ir,iz] = s
           # J[ir, iz] = ((R2-R1)/N1) * (np.pi/N2) * s * 2 /np.pi
    return(Ms)

In [102]:
def vec_4(nr, nz, N1, N2):
    #for test only
    drv = dz = 2
    R1 =15
    R2 =20
    LAMBDA = 5
    
    #code of interest:
    Mr =  drv * (np.arange(nr)+0.5)
    Mz = dz * (np.arange(nz)+0.5)
    Mrc =  R1 + ((R2-R1)/N1) * (np.arange(N1)+0.5) 
    Mfi = (np.pi/N2) * (np.arange(N2)+0.5)
    
    Ms = np.zeros((nr, nz))
    
    Mr00 = Mr.reshape((nr,1,1,1)) * Mrc.reshape((1,1,N1,1))
    Mr01 = Mr00 * np.cos(Mfi).reshape((1,1,1,N2))
    Mr00 = (Mz**2).reshape(1,nz,1,1) - 2 * Mr01
    Mr01 = Mr00 + (Mr**2).reshape((nr,1,1,1)) 
    Mr00 = Mr01 + (Mrc**2).reshape((1,1,N1,1))
    Mr0 = np.sqrt(Mr00)
    
    S0 = np.exp(-Mr0/LAMBDA
               ) / (LAMBDA * np.power(Mr0, 3))
    #print(S0.shape)
    S1 = np.sum(S0, -1)
    #print(S1.shape)
    S0 = np.matmul(S1, Mrc)
    #print(S0.shape)
    S1 = S0 * Mz.reshape(1,nz)
    #print(S1.shape)
                    
    return(S1)

In [104]:
Ms1 =vec_4(2,3,4,5)
Ms0 = for_loops_4(2,3,4,5)
print(np.max(Ms0-Ms1))
print(np.min(Ms0-Ms1))

(2, 3, 4, 5)
(2, 3, 4)
(2, 3)
(2, 3)
2.168404344971009e-19
-4.336808689942018e-19


## 4. final

In [105]:
def vec_fin(nr, nz, N1, N2):
    #for test only
    drv = dz = 2
    R1 =15
    R2 =20
    LAMBDA = 5
    
    #code of interest:
    Mr =  drv * (np.arange(nr)+0.5)
    Mz = dz * (np.arange(nz)+0.5)
    Mrc =  R1 + ((R2-R1)/N1) * (np.arange(N1)+0.5) 
    Mfi = (np.pi/N2) * (np.arange(N2)+0.5)
    
    Ms = np.zeros((nr, nz))
    
    Mr00 = Mr.reshape((nr,1,1,1)) * Mrc.reshape((1,1,N1,1))
    Mr01 = Mr00 * np.cos(Mfi).reshape((1,1,1,N2))
    Mr00 = (Mz**2).reshape(1,nz,1,1) - 2 * Mr01
    Mr01 = Mr00 + (Mr**2).reshape((nr,1,1,1)) 
    Mr00 = Mr01 + (Mrc**2).reshape((1,1,N1,1))
    Mr0 = np.sqrt(Mr00)
    
    S0 = np.exp(-Mr0/LAMBDA
               ) / (LAMBDA * np.power(Mr0, 3))
    #print(S0.shape)
    S1 = np.sum(S0, -1)
    #print(S1.shape)
    S0 = np.matmul(S1, Mrc)
    #print(S0.shape)
    S1 = S0 * Mz.reshape(1,nz)
    #print(S1.shape)
    
    J= (((R2-R1)/N1) * (np.pi/N2) * 2 /np.pi) * S1
                    
    return(J)

#J[ir, iz] = ((R2-R1)/N1) * (np.pi/N2) * s * 2 /np.pi

In [106]:
Jfor = for_loops_1(2,3,4,5)
Jvec = vec_fin(2,3,4,5)

print(np.max(Jfor-Jvec))
print(np.min(Jfor-Jvec))

1.0842021724855044e-19
-2.168404344971009e-19


In [108]:
Jfor = for_loops_1(15,20,10,15)
Jvec = vec_fin(15,20,10,15)

print(np.max(Jfor/Jvec))
print(np.min(Jfor/Jvec))

1.000000000000001
0.9999999999999989


In [109]:
%timeit for_loops_1(15,20,10,15)

599 ms ± 180 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [110]:
%timeit vec_fin(15,20,10,15)

2.58 ms ± 151 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [111]:
x=30
%timeit for_loops_1(x,x,x,x)
%timeit vec_fin(x,x,x,x)

9.75 s ± 1.42 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
55.5 ms ± 2.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
